# Using fNIRS Channel Variance as a Proxy for Measurement Noise

To improve statistics, channel pruning might not always be the way. 
An alternative is to use channel weights in the calculation of averages (e.g. across subjects) or image reconstruction. 
One way of weighting channels is by their estimated measurement noise. 
Variance can be a proxy of measurement noise, e.g. when calculated across trials of the same condition (within subject) or across time on the residual after GLM fit.
This notebook is WIP to provide help to explore this approach with a helper function (quality.measurement_variance) for this purpose.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

import cedalion
import cedalion.datasets as datasets
import cedalion.nirs
import cedalion.sigproc.quality as quality
import cedalion.sigproc.motion_correct as motion_correct
from cedalion import units
import cedalion.xrutils as xrutils

%matplotlib widget

In [ ]:
# some plotting helper functions for this notebook

def plot_heatmap(da, cov_wavelength=None, figsize=(12, 4), cmap=None):
    """ Plot either:
      1. A “variance” DataArray with dims ("channel", "wavelength"), as a channel×wavelength heatmap;
      2. A “covariance” DataArray with dims ("wavelength", "channel1", "channel2"), 
         by selecting exactly one wavelength (cov_wavelength) and plotting the channel1×channel2 matrix.

    Parameters
    ----------
    da : xarray.DataArray
        - If dims == ("channel", "wavelength"): plot variance (channels on x, wavelengths on y).
        - If dims == ("wavelength", "channel1", "channel2"): plot covariance at one wavelength.
    cov_wavelength : scalar, optional
        Required only when da.dims == ("wavelength", "channel1", "channel2"). Must be one of da.wavelength values.
    figsize : tuple of float, optional
        Figure size in inches. Defaults to (12, 4).
    cmap : str or matplotlib.Colormap, optional
        Which colormap to use. If None, uses the default.

    Returns
    -------
    fig, ax : matplotlib.Figure, matplotlib.Axes
        Figure and Axes objects, for any further customization.
    """

    dims = da.dims

    # 1) VARIANCE CASE: dims = ("channel", "wavelength")
    if set(dims) == {"channel", "wavelength"}:
        # Convert to pandas DataFrame so that rows = channels, cols = wavelengths
        df = da.to_pandas()

        # We want channels on the x-axis, wavelengths on the y-axis.
        #   df.values has shape (n_channels, n_wavelengths), so transpose → (n_wavelengths, n_channels)
        arr = df.values.T

        x_labels = df.index.tolist()         # channel names
        y_labels = [str(int(wl)) for wl in df.columns]  # wavelength values as strings

        x_dim_name = "channel"
        y_dim_name = "wavelength"
        cbar_label = "Variance"

    # 2) COVARIANCE CASE: dims = ("wavelength", "channel1", "channel2")
    elif set(dims) == {"wavelength", "channel1", "channel2"}:
        if cov_wavelength is None:
            raise ValueError(
                "When da.dims == ('wavelength','channel1','channel2'), you must supply cov_wavelength."
            )
        # Extract the 2D slice at that wavelength
        da2d = da.sel(wavelength=cov_wavelength)
        # Make sure dims are in order (channel1, channel2)
        da2d = da2d.transpose("channel1", "channel2")

        arr = da2d.values  # shape = (n_channel1, n_channel2)

        x_labels = da2d.coords["channel2"].values.tolist()
        y_labels = da2d.coords["channel1"].values.tolist()

        x_dim_name = "channel2"
        y_dim_name = "channel1"
        cbar_label = f"Covariance (λ={cov_wavelength})"

    else:
        raise ValueError(f"Unsupported DataArray dimensions: {dims}")

    # 3) Plot the 2D array with imshow
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(arr, aspect="auto", cmap=cmap)

    # 4) Set x-axis ticks/labels
    ax.set_xticks(range(len(x_labels)))
    ax.set_xticklabels(x_labels, rotation=90, fontsize=8)

    # 5) Set y-axis ticks/labels
    ax.set_yticks(range(len(y_labels)))
    ax.set_yticklabels(y_labels, fontsize=8)

    # 6) Label axes from the dimension names
    ax.set_xlabel(x_dim_name)
    ax.set_ylabel(y_dim_name)

    # 7) Add a colorbar
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label(cbar_label)

    plt.tight_layout()
    return fig, ax

## Plain Channel Variance
Note: channel variance can only be a proxy for measurement noise if calculated OD or CONC. Do not calculate on raw intensity.

In [ ]:
# get example finger tapping dataset
rec = datasets.get_fingertapping()
rec["od"] = cedalion.nirs.int2od(rec["amp"])


# Plot some data for visual validation
f,ax = plt.subplots(1,1, figsize=(12,4))
ax.plot( rec["od"].time, rec["od"].sel(channel="S1D1", wavelength="850"), "r-", label="S1D1 850nm")
ax.plot( rec["od"].time, rec["od"].sel(channel="S1D1", wavelength="760"), "b-", label="S1D1 760nm")
plt.legend()
ax.set_xlim(0, 500)
ax.set_xlabel("time / s")
ax.set_ylabel("Signal intensity / a.u.")

Calculate variance of all channels and display results

In [ ]:
# calculate variance of optical density (OD) measurements for all channels and wavelengths
od_var = quality.measurement_variance(rec["od"])
display(od_var)

fig, ax = plot_heatmap(od_var)
plt.show()

From the plot above we can identify S6D8 (760nm) as a channel with high variance and and S1D2 (760nm) as a channel with low variance. S7D6 is somewhere inbetween.
Lets investigate how the corresponding time series looks like.

In [ ]:
# Plot some data for visual validation
f,ax = plt.subplots(1,1, figsize=(12,4))
ax.plot( rec["od"].time, rec["od"].sel(channel="S6D8", wavelength="760"), "r-", label="S6D8 760nm")
ax.plot( rec["od"].time, rec["od"].sel(channel="S1D2", wavelength="760"), "b-", label="S1D2 760nm")
ax.plot( rec["od"].time, rec["od"].sel(channel="S7D6", wavelength="760"), "y-", label="S7D6 760nm")
plt.legend()
ax.set_xlim(0, 500)
ax.set_xlabel("time / s")
ax.set_ylabel("Signal intensity / a.u.")

We can see that the Channel with high variance has motion artifacts. These can be removed with [motion correction methods](https://doc.ibs.tu-berlin.de/cedalion/doc/dev/examples/signal_quality/22_motion_artefacts_and_correction.html) and we can recalculate the variance to see if this helped. If we don't, and use the channel variance as is for weighting in further processing, the channel with motion artifacts will be downweighted, as it has higher variance.

In [ ]:
# motion correction using the wavelet and tddr methods  
rec["od_corrected"] = motion_correct.tddr(rec["od"])
rec["od_corrected"] = motion_correct.motion_correct_wavelet(rec["od_corrected"])


# Plot corrected data for visual validation
f,ax = plt.subplots(1,1, figsize=(12,4))
ax.plot( rec["od_corrected"].time, rec["od_corrected"].sel(channel="S6D8", wavelength="760"), "r-", label="S6D8 760nm")
ax.plot( rec["od_corrected"].time, rec["od_corrected"].sel(channel="S1D2", wavelength="760"), "b-", label="S1D2 760nm")
ax.plot( rec["od_corrected"].time, rec["od_corrected"].sel(channel="S7D6", wavelength="760"), "y-", label="S7D6 760nm")
plt.legend()
ax.set_xlim(0, 500)
ax.set_xlabel("time / s")
ax.set_ylabel("Signal intensity / a.u.")

# calculate variance on the corrected signal
od_var2 = quality.measurement_variance(rec["od_corrected"])


## Display results as a heatmap
fig, ax = plot_heatmap(od_var2)
plt.show()

We can see that motion correction took care of some variance (and therefore fixed some channels like S6D8), but not all of it, S4D4 remains partially noisy.

## Channel variance under consideration of flagged bad channels
There are cases in which we don't trust channel variance as a proxy for measurement noise. Examples are saturated channels.
We could  also want to penalize channels with motion artifacts particulalry strongly and for instance kick out S4D4, which did only partially profit from artifact rejection.
For this we can provide a list of "bad" channels and a custom weight.

In [ ]:
# lets assume we do not want to do motion correction and channel S1D1 is saturated. 
# We give S1D1 it a constant value of 1V with only the measurement noise of the system of 10mV
rec["amp"].loc[{"channel": "S1D1"}] = (1 + np.random.normal(0, 10e-3, rec["amp"].sel(channel="S1D1").shape))*units.V

# now convert the signal to optical density
rec["od"] = cedalion.nirs.int2od(rec["amp"])


# Plot some data for visual validation
f,ax = plt.subplots(1,1, figsize=(12,4))
ax.plot( rec["od"].time, rec["od"].sel(channel="S1D1", wavelength="760"), "b-", label="S1D1 760nm")
ax.plot( rec["od"].time, rec["od"].sel(channel="S4D4", wavelength="850"), "r-", label="S4D4 850nm")
plt.legend()
ax.set_xlim(0, 500)
ax.set_xlabel("time / s")
ax.set_ylabel("Signal intensity / a.u.")

Looking at the resulting variance of the saturated channel S1D2 and comparing it with the noisy (motion artifact) channel S4D4...

In [ ]:
# calculate variance of optical density (OD) measurements for all channels and chromophores
od_var = quality.measurement_variance(rec["od"])

# print channel S1D1 760nm and channel S6D8 760nm variance 
print("S1D1 760nm variance:", od_var.sel(channel="S1D1", wavelength="760").values)
print("S4D4 760nm variance:", od_var.sel(channel="S4D4", wavelength="850").values)


we can tell that the metric cannot account for saturation, and we should manually drop / downweight the saturated channel. 

In [ ]:
list_bad_channels = ["S1D1", "S4D4"]
bad_rel_var = 1e5 # we use a large factor that will be multiplied with the channel variance to effectively remove the channel from the analysis wherever it is weighted by its variance

od_var = quality.measurement_variance(rec["od"], list_bad_channels, bad_rel_var)

## Display results as a heatmap, this time on a logarithmic scale as the penalty factor is large
fig, ax = plot_heatmap(np.log(od_var))
plt.show()

## Using Variance as Proxy for measurement Noise to Downweight Channels
Lets apply this now, for instance to normalize signals using the noise proxy (smaller variance will amplify a signal)

In [ ]:
# normalize signals by their variance
rec["normalized_od"] = rec["od"] / od_var

# Plot normalized data for visual validation
f,ax = plt.subplots(1,1, figsize=(12,4))

ax.plot( rec["normalized_od"].time, rec["normalized_od"].sel(channel="S4D12", wavelength="760"), "g-", label="S4D12 760nm, weighted with variance = " +str(od_var.sel(channel="S4D12", wavelength="760").values))
ax.plot( rec["normalized_od"].time, rec["normalized_od"].sel(channel="S1D2", wavelength="760"), "b-", label="S1D2 760nm, weighted with variance = " +str(od_var.sel(channel="S1D2", wavelength="760").values))
ax.plot( rec["normalized_od"].time, rec["normalized_od"].sel(channel="S7D6", wavelength="760"), "y-", label="S7D6 760nm, weighted with variance = " +str(od_var.sel(channel="S7D6", wavelength="760").values))
ax.plot( rec["normalized_od"].time, rec["normalized_od"].sel(channel="S1D1", wavelength="760"), "r-", label="S1D1 760nm, weighted with penalty*variance = "+str(bad_rel_var*od_var.sel(channel="S1D2", wavelength="760").values))
plt.legend()
ax.set_xlim(0, 500)
ax.set_xlabel("time / s")
ax.set_ylabel("Signal intensity / a.u.")

## Channel Covariance
Lastly, we might also be interested in channel covariance...

In [ ]:
# use the same function to calculate the covariance of the optical density measurements
list_bad_channels = ["S1D1", "S4D4"]
bad_rel_var = 10  # much smaller factor than the default just to highlight the effect

od_covar = quality.measurement_variance(rec["od"], list_bad_channels, bad_rel_var, calc_covariance=True)
display(od_covar)


# use log(Var) again because we penalized the bad channels with a large factor
fig, ax = plot_heatmap(od_covar, cov_wavelength=760.0, figsize=(10, 10))
plt.show()